In [119]:
##pip install duckdb --upgrade

In [47]:
import duckdb

## it will create a basic database file
con = duckdb.connect("datacamp.duckdb")

In [49]:
## you should use SQL to create a table, if you want to load csv file(`read.csv`)
con.execute("""
    CREATE TABLE IF NOT EXISTS bank AS 
    SELECT * FROM read_csv('bank-marketing.csv')
""")
con.execute("SHOW ALL TABLES").fetchdf()

,database,schema,name,column_names,column_types,temporary
0,datacamp,main,bank,"[age, job, marital, education, default, housin...","[BIGINT, VARCHAR, VARCHAR, VARCHAR, VARCHAR, V...",False
1,datacamp,main,blog,"[node_id, text, embedding, metadata_]","[VARCHAR, VARCHAR, FLOAT[1536], JSON]",False
2,datacamp,main,gpa,"[node_id, text, embedding, metadata_]","[VARCHAR, VARCHAR, FLOAT[1536], JSON]",False


In [51]:
con.execute("SELECT * FROM bank WHERE duration < 100 LIMIT 10").fetchdf()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,25,services,single,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,False
1,41,blue-collar,married,unknown,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,False
2,30,unemployed,married,high.school,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,False
3,35,technician,married,university.degree,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,False
4,59,technician,married,unknown,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,False
5,49,blue-collar,married,unknown,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,False
6,42,technician,single,professional.course,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,False
7,44,blue-collar,single,basic.9y,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,False
8,38,technician,single,university.degree,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,False
9,57,unknown,married,unknown,unknown,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,False


In [53]:
bank_duck = duckdb.read_csv("bank-marketing.csv",sep=";")
bank_duck.filter("duration < 100").limit(3).df()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,25,services,single,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,41,blue-collar,married,unknown,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,30,unemployed,married,high.school,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


In [55]:
rel = con.table("bank")
rel.columns

['age',
 'job',
 'marital',
 'education',
 'default',
 'housing',
 'loan',
 'contact',
 'month',
 'day_of_week',
 'duration',
 'campaign',
 'pdays',
 'previous',
 'poutcome',
 'emp.var.rate',
 'cons.price.idx',
 'cons.conf.idx',
 'euribor3m',
 'nr.employed',
 'y']

In [57]:
rel.filter("duration < 100").project("job,education,loan").order("job").limit(3).df()

,job,education,loan
0,admin.,university.degree,no
1,admin.,high.school,yes
2,admin.,high.school,no


In [59]:
res = duckdb.query("""SELECT 
                            job,
                            COUNT(*) AS total_clients_contacted,
                            AVG(duration) AS avg_campaign_duration,
                        FROM 
                            'bank-marketing.csv'
                        WHERE 
                            age > 30
                        GROUP BY 
                            job
                        ORDER BY 
                            total_clients_contacted DESC;""")
res.df()

,job,total_clients_contacted,avg_campaign_duration
0,admin.,8276,252.960488
1,blue-collar,7763,263.795955
2,technician,5578,249.012370
3,services,3054,256.742305
4,management,2658,257.012792
5,retired,1715,273.891545
6,entrepreneur,1318,256.918816
7,self-employed,1161,267.500431
8,housemaid,1001,247.519481
9,unemployed,845,251.962130


In [61]:
%%capture
%pip install duckdb
%pip install llama-index
%pip install llama-index-vector-stores-duckdb

In [62]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.vector_stores.duckdb import DuckDBVectorStore
from llama_index.core import StorageContext
from IPython.display import Markdown, display

In [73]:
## import llm model from OPENAI(GPT4o-mini)
import os
from dotenv import load_dotenv
from llama_index.llms.openai import OpenAI
load_dotenv()

llm = OpenAI(model="gpt-4o-mini",api_key=os.environ["OPENAI_API_KEY"])

In [75]:
## create a text embedding models 
from llama_index.embeddings.openai import OpenAIEmbedding
embed_model = OpenAIEmbedding(
    model="text-embedding-3-small",
)

In [77]:
## config LlamaIndex setting
from llama_index.core import Settings
Settings.llm = llm
Settings.embed_model = embed_model

In [79]:
## Define the path of files which need to read 
documents = SimpleDirectoryReader("Data").load_data()

In [81]:
## Split the document into nodes
## A vector embedding is generated for each node
## Store the embedding and the corresponding text in the specified vector store

#database name, table name, persistence directory, and embedding vector dimensions are set
vector_store = DuckDBVectorStore(database_name = "datacamp.duckdb",table_name = "pdf_file",persist_dir="./", embed_dim=1536)
#The custom vector_store is added to the storage context
storage_context = StorageContext.from_defaults(vector_store=vector_store)

index = VectorStoreIndex.from_documents(documents, storage_context=storage_context)

In [83]:
## connect to the datacamp.duckdb database
import duckdb
con = duckdb.connect("datacamp.duckdb")
con.execute("SHOW ALL TABLES").fetchdf()

,database,schema,name,column_names,column_types,temporary
0,datacamp,main,bank,"[age, job, marital, education, default, housin...","[BIGINT, VARCHAR, VARCHAR, VARCHAR, VARCHAR, V...",False
1,datacamp,main,blog,"[node_id, text, embedding, metadata_]","[VARCHAR, VARCHAR, FLOAT[1536], JSON]",False
2,datacamp,main,gpa,"[node_id, text, embedding, metadata_]","[VARCHAR, VARCHAR, FLOAT[1536], JSON]",False


In [85]:
##creates a default query engine for the given index. 
#This query engine allows you to ask questions about the indexed document(Our Pdf file in the Data folder)
query_engine = index.as_query_engine()
response = query_engine.query("How to use Flask?")
display(Markdown(f"<b>{response}</b>"))

<b>To use Flask, start by installing it and setting up your environment. You can follow the installation instructions provided in the documentation. Once installed, you can create a minimal application by defining routes and view functions. Utilize the Quickstart section for a straightforward introduction, and refer to the detailed Tutorial for a comprehensive guide on building a complete application. Additionally, explore the User's Guide for information on configuration, routing, and handling static files. For more advanced features, check out the Useful Internals section, which covers topics like class-based views and the command line interface.</b>

In [91]:
## ask something not exist in Data folder
response = query_engine.query("do u know Tian's gpa")
display(Markdown(f"<b>{response}</b>"))

<b>There is no information available regarding Tian's GPA.</b>

In [93]:
## Using CondensePlusContextChatEngine to keep the conversation history
## Be able to retrieve context

from llama_index.core.memory import ChatMemoryBuffer
from llama_index.core.chat_engine import CondensePlusContextChatEngine

## create chat memory buffer
memory = ChatMemoryBuffer.from_defaults(token_limit=3900)

chat_engine = CondensePlusContextChatEngine.from_defaults(
    index.as_retriever(),
    memory=memory,
    llm=llm
)

response = chat_engine.chat(
    "tell me something about duckDB."
)

display(Markdown(response.response))

DuckDB is a relational database management system (DBMS) that is designed to be simple, fast, and feature-rich. It supports the Structured Query Language (SQL) and is particularly focused on analytical use cases. Here are some key points about DuckDB:

1. **Simplicity**: DuckDB is designed for ease of installation and use. It has no external dependencies, which means that it can be compiled into just two files—a header and an implementation file—making deployment straightforward. All you need to build DuckDB is a working C++11 compiler.

2. **Embedded Operation**: Unlike traditional DBMS that run as separate server processes, DuckDB operates completely embedded within a host process. This design choice allows for high-speed data transfer to and from the database, and in some cases, it can process foreign data without the need for copying. For instance, the DuckDB Python package can execute queries directly on Pandas data without importing or copying it.

3. **Portability**: DuckDB is highly portable and can be compiled for all major operating systems (Linux, macOS, Windows) and CPU architectures (x86, ARM). It can run on a wide range of devices, from small edge devices to large multi-terabyte memory servers. Additionally, with DuckDB-Wasm, it can even run in web browsers and on mobile phones.

4. **Feature-Rich**: DuckDB offers extensive support for complex SQL queries, including a large function library and window functions. It also provides transactional guarantees (ACID properties), ensuring data integrity and reliability.

5. **APIs**: DuckDB provides APIs for various programming languages, including Java, C, C++, Go, and Node.js, making it versatile for developers working in different environments.

Overall, DuckDB is a powerful tool for data management, especially suited for analytical tasks, while maintaining a focus on simplicity and performance.

In [95]:
response = chat_engine.chat(
    "Could you please provide more details about the duckDB?"
)
display(Markdown(response.response))

Certainly! Here are more detailed aspects of DuckDB:

1. **Architecture**: DuckDB is designed as an embedded database, meaning it runs within the application process rather than as a standalone server. This architecture allows for faster data access and manipulation since it eliminates the overhead of network communication. This is particularly beneficial for analytical workloads where speed is crucial.

2. **Performance**: DuckDB is optimized for analytical queries and can handle large datasets efficiently. It employs various optimization techniques, such as vectorized execution, which processes data in batches rather than one row at a time, significantly improving performance. Additionally, DuckDB has been tested under heavy loads using benchmarks like TPC-H and TPC-DS, ensuring it can handle demanding analytical tasks.

3. **Data Handling**: One of DuckDB's standout features is its ability to work with data directly from other sources without the need for data copying. For example, when using the DuckDB Python package, users can run SQL queries directly on Pandas DataFrames. This capability allows for seamless integration with data science workflows, making it easier to analyze data without the overhead of data transfer.

4. **SQL Support**: DuckDB supports a rich set of SQL features, including complex queries, window functions, and a large library of built-in functions. This makes it suitable for a wide range of analytical tasks, from simple aggregations to complex data transformations.

5. **ACID Compliance**: DuckDB provides ACID (Atomicity, Consistency, Isolation, Durability) guarantees, which are essential for ensuring data integrity in transactional applications. This means that even in the event of a failure, DuckDB can maintain a consistent state of the database.

6. **Extensibility**: DuckDB is designed to be extensible, allowing developers to add custom functions and features as needed. This flexibility makes it a good choice for specialized applications that may require unique data processing capabilities.

7. **Cross-Platform Compatibility**: DuckDB can be compiled for various operating systems and architectures, making it highly portable. It can run on everything from small edge devices to powerful servers with multiple CPU cores. The introduction of DuckDB-Wasm allows it to run in web browsers, expanding its usability even further.

8. **Community and Research**: DuckDB has a growing community and is backed by research, with several peer-reviewed papers and thesis works published on its architecture and capabilities. This research foundation helps ensure that DuckDB remains at the forefront of database technology.

9. **Documentation and Support**: DuckDB comes with comprehensive documentation that covers installation, configuration, data import, and usage guides. This makes it easier for new users to get started and for experienced users to find advanced features.

In summary, DuckDB is a powerful, flexible, and efficient embedded analytical database that is well-suited for modern data analysis tasks, particularly in data science and analytics environments. Its ease of use, performance, and rich feature set make it an attractive choice for developers and data analysts alike.

In [97]:
%pip install duckdb-engine -q

Note: you may need to restart the kernel to use updated packages.


In [99]:
from sqlalchemy import create_engine

engine = create_engine("duckdb:///datacamp.duckdb")
##Execute native SQL queries
with engine.connect() as connection:
    cursor = connection.exec_driver_sql("SELECT * FROM bank LIMIT 3")
    print(cursor.fetchall())

[(56, 'housemaid', 'married', 'basic.4y', 'no', 'no', 'no', 'telephone', 'may', 'mon', 261, 1, 999, 0, 'nonexistent', 1.1, 93.994, -36.4, 4.857, 5191.0, False), (57, 'services', 'married', 'high.school', 'unknown', 'no', 'no', 'telephone', 'may', 'mon', 149, 1, 999, 0, 'nonexistent', 1.1, 93.994, -36.4, 4.857, 5191.0, False), (37, 'services', 'married', 'high.school', 'no', 'yes', 'no', 'telephone', 'may', 'mon', 226, 1, 999, 0, 'nonexistent', 1.1, 93.994, -36.4, 4.857, 5191.0, False)]


In [117]:
## SQLDatabase provides an interface for LlamaIndex to interact with the underlying SQL database.
## Allows other components of LlamaIndex, such as the query engine, 
## to understand and manipulate the structure and content of the SQL database.

from llama_index.core import SQLDatabase
sql_database = SQLDatabase(engine, include_tables=["bank"])

In [107]:
## NLSQLTableQueryEngine, which translates natural language queries into SQL queries 
## And executes them in database
from llama_index.core.query_engine import NLSQLTableQueryEngine

query_engine = NLSQLTableQueryEngine(sql_database)

In [109]:
response = query_engine.query("Which is the longest running campaign?")

print(response.response)

The longest running campaign is campaign number 1, with a duration of 4918 days.


In [84]:
response = query_engine.query("Which type of job has the most housing loan?")
print(response.response)

The type of job with the most housing loans is "admin," with a total of 5,559 housing loans recorded.


In [86]:
print(response.metadata)

{'35317a2e-ab8b-4391-ae35-11c07aa4d0a6': {'sql_query': "SELECT job, COUNT(*) AS housing_loan_count FROM bank WHERE housing = 'yes' GROUP BY job ORDER BY housing_loan_count DESC LIMIT 1", 'result': [('admin.', 5559)], 'col_keys': ['job', 'housing_loan_count']}, 'sql_query': "SELECT job, COUNT(*) AS housing_loan_count FROM bank WHERE housing = 'yes' GROUP BY job ORDER BY housing_loan_count DESC LIMIT 1", 'result': [('admin.', 5559)], 'col_keys': ['job', 'housing_loan_count']}


In [113]:
response = query_engine.query("how many users no married ?")
print(response.response)

There are 16,260 users who are not married.
